<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MYVERSION_OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/tools/OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OnDemandLoaderTool Tutorial

Our `OnDemandLoaderTool` is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.

This tool takes in a `BaseReader` data loader, and when called will 1) load data, 2) index data, and 3) query the data.

In this walkthrough, we show how to use the `OnDemandLoaderTool` to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-readers-wikipedia

In [ ]:
!pip install llama-index

In [3]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

### Define Tool

We first define the `WikipediaReader`. Note that the `load_data` interface to `WikipediaReader` takes in a list of `pages`. By default, this queries the Wikipedia search endpoint which will autosuggest the relevant pages.

We then wrap it into our `OnDemandLoaderTool`.

By default since we don't specify the `index_cls`, a simple vector store index is initialized.

In [ ]:
!pip install wikipedia
!pip install langchain

In [5]:
reader = WikipediaReader()

In [6]:
tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

#### Testing

We can try running the tool by itself (or as a LangChain tool), just to showcase what the interface is like!

Note that besides the arguments required for the data loader, the tool also takes in a `query_str` which will be
the query against the index.

In [ ]:
#added by Frank Morales(FM) 22/02/2024
%pip install openai  --root-user-action=ignore
%pip install colab-env --upgrade --quiet --root-user-action=ignore

# OpenAI Settings

In [ ]:
#added by Frank Morales(FM) 22/02/2024
import warnings
warnings.filterwarnings('ignore')

import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()

In [ ]:
# run tool by itself
#added by Frank Morales(FM) 22/02/2024
tool(["Montreal"], query_str="What is the best restaurant in Montreal?")

In [10]:
# run tool as langchain structured tool
lc_tool = tool.to_langchain_structured_tool(verbose=True)

In [ ]:
lc_tool.run(
    tool_input={
        "pages": ["Montreal"],
        "query_str": "What is the best restaurant in Montreal?",
    }
)

### Initialize LangChain Agent

For tutorial purposes, the agent just has access to one tool - the Wikipedia Reader

Note that we need to use Structured Tools from LangChain.

In [19]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

In [18]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)

In [17]:
agent = initialize_agent(
    [lc_tool],
    llm=llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=False,
)

# Now let's run some queries!

The OnDemandLoaderTool allows the agent to simultaneously 1) load the data from Wikipedia, 2) query that data.

In [24]:
agent.run("What is the best restaurant in Montreal?")


Observation: Au Pied de Cochon, La Banquise, La Binerie Mont-Royal, Boustan, Casa del Popolo, Chenoy's, Le Cheval Blanc, Cuisine AuntDai, Décarie Hot Dogs, Dunn's, Eaton's Ninth Floor Restaurant, Gibeau Orange Julep, J&R Kosher Meat and Delicatessen, Joe Beef, Restaurant Pizzaiolle, Mikes, Moishes Steakhouse, Montreal Pool Room, Schwartz's, Toqué!, Wilensky's
Thought:

"Some of the best restaurants in Montreal are Au Pied de Cochon, La Banquise, Joe Beef, Schwartz's, and Toqué!"